In [2]:
# 小范围测试代码bug原因

In [3]:
# 时间格式有影响："2019-08-06 10:25:50.000" 正常运行
import pandas as pd
df = pd.DataFrame({
"timestamps":["2019-08-06 10:25:50.000"],
"y":[1] })
df['timestamps'] = df['timestamps'].astype('datetime64[ms]')

In [4]:
# 时间格式有影响："2019-08-06 10:25:50:000" 报错
# ParserError: Unknown string format: 2019-08-06 10:25:50:000
import pandas as pd
df = pd.DataFrame({
"timestamps":["2019-08-06 10:25:50:000"],
"y":[1] })
df['timestamps'] = df['timestamps'].astype('datetime64[ms]')

ParserError: Unknown string format: 2019-08-06 10:25:50:000

In [6]:
import pandas as pd
df = pd.DataFrame({
"timestamps":["2019-08-06 10:25:50:000"],
"y":[1] })
ts = df["timestamps"]
timestamps = pd.DatetimeIndex(ts.astype('datetime64[ms]'))

ParserError: Unknown string format: 2019-08-06 10:25:50:000

In [7]:
import pandas as pd
df = pd.DataFrame({
"timestamps":["2019-08-06 10:25:50.000"],
"y":[1] })
ts = df["timestamps"]
timestamps = pd.DatetimeIndex(ts.astype('datetime64[ms]'))
timestamps

DatetimeIndex(['2019-08-06 10:25:50'], dtype='datetime64[ns]', name='timestamps', freq=None)

In [8]:
import pandas as pd
import re
df = pd.DataFrame({
"timestamps":["2019-08-06 10:25:50.000"],
"y":[1] })
df["timestamps"] = df["timestamps"].apply(lambda x: re.sub(r':(\d{3})$', r'.\1', x))
df["timestamps"]

0    2019-08-06 10:25:50.000
Name: timestamps, dtype: object

In [9]:

import os
import pandas as pd
import re
path = "/Users/wangshuaibo/Documents/ScriptsofShuai/gaitpy_exercise/"
file = "demo_data_modified.csv"
os.chdir(path)
df = pd.read_csv(file,dtype={"timestamps": str})
df["timestamps"] = df["timestamps"].apply(lambda x: re.sub(r':(\d{3})$', r'.\1', x))
df.to_csv("demo_data_modified_time.csv",index=False)
#  以上代码解决了第一个报错

# 以下代码复现了第二个报错
df1 = pd.read_csv("demo_data_modified_time.csv")
timestamps = df["timestamps"]
type(timestamps[0])
start_time = timestamps[0].astype('datetime64[ms]')


AttributeError: 'str' object has no attribute 'astype'

In [14]:
# 更改下顺序就不报错


import os
import pandas as pd
import re
path = "/Users/wangshuaibo/Documents/ScriptsofShuai/gaitpy_exercise/"
file = "demo_data_modified.csv"
os.chdir(path)
df = pd.read_csv(file,dtype={"timestamps": str})
df["timestamps"] = df["timestamps"].apply(lambda x: re.sub(r':(\d{3})$', r'.\1', x))
df.to_csv("demo_data_modified_time.csv",index=False)
#  以上代码解决了第一个报错

# 以下代码复现了第二个报错
df1 = pd.read_csv("demo_data_modified_time.csv")
timestamps = df["timestamps"]
type(timestamps[0])

start_time = timestamps.astype('datetime64[ms]')[0]
end_time = timestamps.astype('datetime64[ms]').iloc[-1]
gait_bouts = pd.DataFrame(data={'start_time': [start_time],
                                'end_time': [end_time],
                                'bout_length': [(end_time - start_time).total_seconds()]})


修改内容

start_time = timestamps[0].astype('datetime64[ms]')

end_time = timestamps.iloc[-1].astype('datetime64[ms]')

修改为

start_time = timestamps.astype('datetime64[ms]')[0]

 end_time = timestamps.astype('datetime64[ms]').iloc[-1]

修改

'bout_length': [(end_time - start_time).item().total_seconds()]})

修改为

'bout_length': [(end_time - start_time).total_seconds()]})